In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# Load the dataset
df = pd.read_csv('books.csv', on_bad_lines='skip')

In [3]:
# Handle missing values
df['publisher'].fillna('Unknown', inplace=True)
df['authors'].fillna('Unknown', inplace=True)

C:\Users\devej\AppData\Local\Temp\ipykernel_21172\2469466888.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['publisher'].fillna('Unknown', inplace=True)
C:\Users\devej\AppData\Local\Temp\ipykernel_21172\2469466888.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exam

In [4]:
# Feature Engineering: Combining relevant text features
df['combined_features'] = df['title'].astype(str) + " " + df['authors'].astype(str) + " " + df['publisher'].astype(str)

In [5]:
# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df['combined_features'])

In [6]:
# Compute Cosine Similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [16]:
def recommend_books(title, cosine_sim=cosine_sim, df=df):
    try:
        # Get the index of the book that matches the title
        idx = df[df['title'] == title].index[0]
    except IndexError:
        print(f"Book '{title}' not found.")
        return None  # Exit if the book is not found

    # Get the pairwise similarity scores
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the books based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar books (excluding the book itself)
    sim_scores = sim_scores[1:11]

    # Get the book indices
    book_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar books
    return df['title'].iloc[book_indices]
